In [ ]:
#questo
%%cython
import numpy as np
cimport numpy as np
from libc.stdint cimport int32_t
import time

cdef inline double sign(double x):
    if x > 0:
        return 1.0
    elif x < 0:
        return -1.0
    else:
        return 0.0

def do_some_training_EN(
    URM_train,
    double initial_learning_rate,
    double regularization_2,
    double l1_ratio,
    double decay_rate,
    int num_iterations,
    double[:, :] existing_item_item_S,
    double[:, :] S_icm,     # New parameter
    double alpha           # New parameter
):
    cdef int n_items = URM_train.shape[1]
    URM_train_csr = URM_train.tocsr()
    URM_train_coo = URM_train.tocoo()
    cdef long start_time = time.time()
    cdef int32_t[:] indices = URM_train_csr.indices.view(dtype=np.int32)
    cdef int32_t[:] indptr = URM_train_csr.indptr.view(dtype=np.int32)
    cdef double[:] data = URM_train_csr.data.view(dtype=np.float64)
    cdef int32_t[:] coo_row = URM_train_coo.row.view(dtype=np.int32)
    cdef int32_t[:] coo_col = URM_train_coo.col.view(dtype=np.int32)
    cdef double[:] coo_data = URM_train_coo.data.view(dtype=np.float64)

    cdef double[:, :] item_item_S
    if existing_item_item_S is not None:
        item_item_S = existing_item_item_S
    else:
        item_item_S = np.zeros((n_items, n_items), dtype=np.float64)

    cdef double learning_rate = initial_learning_rate
    cdef double loss = 0.0
    cdef double prediction_error, predicted_rating, profile_rating
    cdef int user_id, item_id, profile_item_id, sample_index, index
    cdef int start_idx, end_idx
    cdef int32_t[:] random_indices = np.random.randint(0, URM_train_coo.nnz, size=num_iterations).astype(np.int32)
     # Early stopping variables
    cdef int patience_counter = 0
    cdef double last_loss = np.inf
    cdef int patience = 20
    cdef double min_delta = 1e-5
    cdef int warm_restart = 0
    for sample_num in range(num_iterations+1):
        sample_index = random_indices[sample_num]
        user_id = coo_row[sample_index]
        item_id = coo_col[sample_index]
        true_rating = coo_data[sample_index]

        predicted_rating = 0.0
        start_idx = indptr[user_id]
        end_idx = indptr[user_id + 1]

        for index in range(start_idx, end_idx):
            profile_item_id = indices[index]
            profile_rating = data[index]
            predicted_rating += profile_rating * item_item_S[profile_item_id, item_id]

        prediction_error = true_rating - predicted_rating
        loss += prediction_error ** 2

        for index in range(start_idx, end_idx):
            profile_item_id = indices[index]
            profile_rating = data[index]
            item_item_S[profile_item_id, item_id] += learning_rate * (
                prediction_error * profile_rating
                - (1 - l1_ratio) * regularization_2 * item_item_S[profile_item_id, item_id]
                - l1_ratio * sign(item_item_S[profile_item_id, item_id])
                - alpha * (item_item_S[profile_item_id, item_id] - S_icm[profile_item_id, item_id]) #new for icm
            )
        if sample_num % 5000 == 0:
            learning_rate *= decay_rate
            if sample_num > 0:
              current_loss = loss / sample_num
            else:
              current_loss=0
            #loss_history.append(current_loss)
            if sample_num % 1000000 == 0:
                  elapsed_time = time.time() - start_time
                  samples_per_second = sample_num / elapsed_time
                  print(
                      "Iteration {} in {:.2f} seconds, loss is {:.4f}. Samples per second {:.2f}".format(
                          sample_num, elapsed_time, current_loss, samples_per_second
                      )
                  )
            # Early stopping check
            if abs(last_loss - current_loss) < min_delta or current_loss > 1:
                patience_counter += 1
                if patience_counter >= patience:
                    if warm_restart == 0:
                        learning_rate = initial_learning_rate
                        warm_restart += 1
                        patience_counter = 0
                        print("warm restart")
                    else:
                        print("Early stopping at iteration {}. Loss has not improved significantly for {} iterations, or has stayed above 1 too much. loss was {}".format(sample_num, patience * 5000, current_loss))
                        break
            else:
                patience_counter = 0  # Reset patience counter if loss improves
            last_loss = current_loss

    return loss, item_item_S


performance hint: /root/.cache/ipython/cython/_cython_magic_793d7496f50a3060672d684fead44deb3ce1ba1a.pyx:56:38: Index should be typed for more efficient access


Content of stderr:
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /root/.cache/ipython/cython/_cython_magic_793d7496f50a3060672d684fead44deb3ce1ba1a.c:1250:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~